In [1]:
import yfinance as yf
import pandas as pd

# Download Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# Reset index
tesla_data.reset_index(inplace=True)

# Display first five rows
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, "html.parser")

# Find all tables
tables = soup.find_all("table")

# Tesla revenue is in the first table
tesla_table = tables[0]

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tesla_table.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace(",", "").replace("$", "")
        tesla_revenue = pd.concat([tesla_revenue,
                                   pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                                  ignore_index=True)

# Clean up data
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype(int)

# Display last 5 rows
tesla_revenue.tail()

,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


In [4]:
# Download GME stock data
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

# Reset index
gme_data.reset_index(inplace=True)

# Display first five rows
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [7]:
# GME revenue is in the second table
gme_table = tables[1]

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in gme_table.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace(",", "").replace("$", "")
        gme_revenue = pd.concat([gme_revenue,
                                 pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                                ignore_index=True)

# Clean up data
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(int)

# Sort values by Date so the latest is at the bottom
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])
gme_revenue = gme_revenue.sort_values("Date")

# Reset index for neatness
gme_revenue.reset_index(drop=True, inplace=True)

# Display last 5 rows (now showing the most recent revenue data)
gme_revenue.tail()

,Date,Revenue
57,2019-04-30,1548
58,2019-07-31,1286
59,2019-10-31,1439
60,2020-01-31,2194
61,2020-04-30,1021


In [8]:
import plotly.graph_objects as go

def make_graph(stock_data, revenue_data, stock_name):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=stock_data.Date, y=stock_data.Close, name="Stock Price"))
    fig.add_trace(go.Bar(x=revenue_data.Date, y=revenue_data.Revenue, name="Revenue", yaxis="y2"))

    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue",
        yaxis=dict(title="Stock Price"),
        yaxis2=dict(title="Revenue", overlaying="y", side="right")
    )

    fig.show()

# Tesla graph
make_graph(tesla_data, tesla_revenue, "Tesla")

In [9]:
# GME graph
make_graph(gme_data, gme_revenue, "GameStop")